# 作業

實作模糊與邊緣檢測

- 透過 Gaussian Filter 實作模糊操作
- 透過 Sobel Filter 實作邊緣檢測

In [1]:
import cv2
import numpy as np

img = cv2.imread('data/lena.png')

## 邊緣檢測

### 比較 Sobel 如果在 uint8 的情況下做會 overflow 的狀況

In [2]:
# 轉為灰階圖片
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 對 x 方向以包含負數的資料格式 (cv2.CV_16S) 進行 Sobel 邊緣檢測
img_sobel_x = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=0, ksize=3)
# 對 x 方向依照比例縮放到所有數值都是非負整數
img_sobel_x = cv2.convertScaleAbs(img_sobel_x)

# 對 x 方向直接以非負整數的資料格式 (uint8) 進行 Sobel 邊緣檢測
# CV_8U,CV_16U,CV_16S,CV_32S,CV_32F,CV_64F
img_sobel_x_uint8 = cv2.Sobel(img_grey, -1 , dx=1, dy=0, ksize=3)

#　組合 + 顯示圖片
img_show = np.hstack((img_grey, img_sobel_x, img_sobel_x_uint8))
while True:
    # 比較 Sobel 邊緣檢測的過程中針對負數操作的不同產生的差異
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

### 比較一次與兩次計算偏微分的結果

In [3]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 求一次導數取得邊緣檢測結果
img_sobel_x = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=0, ksize=3)
img_sobel_x = cv2.convertScaleAbs(img_sobel_x)

# 求二次導數取得邊緣檢測結果
img_sobel_xx = cv2.Sobel(img_grey, cv2.CV_16S, dx=2, dy=0, ksize=3)
img_sobel_xx = cv2.convertScaleAbs(img_sobel_xx)

#　組合 + 顯示圖片
img_show = np.hstack((img_grey, img_sobel_x, img_sobel_xx))
while True:
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 取得 SIFT 特徵
#### pip install opencv-contrib-python==3.4.2.16

In [4]:
# 轉灰階圖片
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 建立 SIFT 物件
SIFT_detector = cv2.xfeatures2d.SIFT_create()

# 取得 SIFT 關鍵點位置
keypoints = SIFT_detector.detect(img_gray, None)

#　畫圖 + 顯示圖片
img_show = cv2.drawKeypoints(img_gray, keypoints, img)
while True:
    cv2.imshow('SIFT', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

In [14]:
# 将KeyPoint格式数据中的xy坐标提取出来
cv2.KeyPoint_convert(keypoints)

array([[ 20.486666,  74.56659 ],
       [ 20.486666,  74.56659 ],
       [ 61.064926, 496.00064 ],
       ...,
       [495.12134 ,  11.863521],
       [495.80933 ,  79.96113 ],
       [495.80933 ,  79.96113 ]], dtype=float32)

## 基於 SIFT 特徵的暴力比對

- 透過 SIFT 特徵實作 Brute-Force Matching
- D.Lowe ratio test
- knn 比對

In [10]:
img_query = cv2.imread('data/box.png', 0)
img_train = cv2.imread('data/box_in_scene.png', 0)

# 建立 SIFT 物件
sift = cv2.xfeatures2d_SIFT.create() # SURF,ORB, BRIEF,BRISK
#sift=cv2.ORB.create()

# 偵測並計算 SIFT 特徵 (keypoints 關鍵點, descriptor 128 維敘述子)
kp_query, des_query = sift.detectAndCompute(img_query, None)
kp_train, des_train = sift.detectAndCompute(img_train, None)

# 建立 Brute-Force Matching 物件
bf = cv2.BFMatcher(cv2.NORM_L2)  # cv2.NORM_L1,cv2.NORM_HAMMING,cv2.NORM_HAMMING2
#bf = cv2.BFMatcher(cv2.NORM_HAMMING)

# 以 knn 方式暴力比對特徵
matches_knn = bf.knnMatch(des_query, des_train, k=2)
matches = bf.match(des_query,des_train)

# 透過 D.Lowe ratio test 排除不適合的配對
candidate_knn = []
for m, n in matches_knn:
    if m.distance < 0.7*n.distance:
        candidate_knn.append([m])
matches=sorted(matches,key=lambda x:x.distance)
candidate=matches[:len(candidate_knn)]

# 顯示配對結果
img_show_knn = cv2.drawMatchesKnn(img_query, kp_query, img_train, kp_train, candidate_knn, None, flags=2)
img_show = cv2.drawMatches(img_query, kp_query, img_train, kp_train, candidate, None, flags=2)
img_show=np.vstack((img_show_knn,img_show))
# 顯示圖片
while True:
    cv2.imshow('matches', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break